# Phase 2: Temporal Analysis (HDFS Version)
This notebook analyzes genre growth trends from 2010 to 2025 using HDFS data.
It also analyzes the popularity of different **Reading Formats** (E-Book, Audio, etc.) over time.

**Update**: Analysis is based on **Read Date** (`read_at`), not Publication Year.
**Optimization**: Processing is split into 5-year intervals to manage memory usage.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, year, when, to_timestamp

# Set Plotting Style
sns.set_theme(style="whitegrid")

# Configurable Limit (Set to 0 for full dataset)
LIMIT = 0

In [2]:
# Initialize Spark (Cluster Mode)
spark = SparkSession.builder \
    .appName("Goodreads_EDA_Temporal") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()

print("✅ Spark Session created.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/14 23:39:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark Session created.


In [3]:
# HDFS Paths
hdfs_base = "hdfs:///user/ubuntu/goodreads_data/processed"
interactions_path = f"{hdfs_base}/master_interactions"

In [4]:
# Read Master Interactions
print("⏳ Reading master_interactions from HDFS...")
try:
    df = spark.read.parquet(interactions_path)
    if LIMIT > 0:
        df = df.limit(LIMIT)
    
    # Parse Dates Globally (this is a narrow transformation, cheap)
    df_read = df.filter((col("is_read") == 1) & (col("read_at").isNotNull()))
    df_parsed = df_read.withColumn("read_date", to_timestamp(col("read_at"), "EEE MMM dd HH:mm:ss Z yyyy"))
    df_with_year = df_parsed.withColumn("read_year", year(col("read_date")))
    
    # Persist the base dataframe with years to avoid re-reading/re-parsing for each interval
    # Using a cache might be heavy if dataset is huge, but filtering `is_read` reduces it significantly.
    # df_with_year.cache()
    print(f"✅ Data loaded.")

except Exception as e:
    print(f"❌ Error: {e}")

⏳ Reading master_interactions from HDFS...


✅ Data loaded.


### Helper Functions for Processing

In [5]:
def process_genre_interval(df_base, start_year, end_year):
    print(f"\n⏳ Processing Interval: {start_year}-{end_year}")
    
    # Filter for Interval
    df_filtered = df_base.filter(
        (col("read_year") >= start_year) & 
        (col("read_year") <= end_year)
    )
    
    # Explode Shelves (Expensive Step)
    df_exploded = df_filtered.select(
        col("read_year"), 
        explode(col("popular_shelves")).alias("shelf")
    )
    
    df_raw_shelves = df_exploded.select(
        col("read_year"),
        col("shelf.name").alias("shelf_name")
    )
    
    # Filter Instructions
    ignore_shelves = [
        'to-read', 'currently-reading', 'favorites', 'books-i-own', 
        'kindle', 'owned', 'ebook', 'library', 'to-buy', 'owned-books', 
        'audio', 'audiobook', 'ebooks', 'audiobooks', 'default', 'wish-list', 
        'my-books', 'read', 'borrowed', 'book-club', 'bookclub', 'book-group',
        'read-in-2010', 'read-in-2011', 'read-in-2012', 'read-in-2013', 'read-in-2014', 'read-in-2015',
        'read-2010', 'read-2011', 'read-2012', 'read-2013', 'read-2014', 'read-2015',
        'dnf', 'did-not-finish', 'abandoned', 'unfinished', 'stopped-reading', 'didn-t-finish',
        'adult', 'adult-fiction', 'novels', 'novel', 'books', 'hardcover', 'paperback',
        'series', 'trilogy', 'standalone', 're-read', 'reread', 'all-time-favorites',
        '5-stars', '4-stars', 'recommended', 'reviewed', 'netgalley', 'arc',
        'kindle-books', 'nook', 'audible', 'calibre', 'shelfari-favorites', 'favorites', 'favourites',
        'owned-tbr', 'tbr', 'to-be-read', 'my-library', 'library-books', 
        'free', 'freebie', 'giveaways', 'kindle-freebie', 'listened-to',
        'english', 'fiction', 'general-fiction', 'literature', 'e-book', 'e-books', 'maybe',
        'own-it', 'on-hold', 'contemporary', 'i-own', 'american', 'favorite'
    ]

    df_genres = df_raw_shelves.filter(~col("shelf_name").isin(ignore_shelves))
    df_genres = df_genres.filter(~col("shelf_name").contains("read-in"))
    df_genres = df_genres.filter(~col("shelf_name").contains("read-20"))
    df_genres = df_genres.filter(~col("shelf_name").contains("to-read"))
    df_genres = df_genres.filter(~col("shelf_name").contains("challenge"))
    df_genres = df_genres.filter(~col("shelf_name").contains("kindle"))
    df_genres = df_genres.filter(~col("shelf_name").contains("audio"))
    df_genres = df_genres.filter(~col("shelf_name").contains("owned"))

    # Normalize
    df_genres = df_genres.withColumn("genre", 
        when(col("shelf_name") == "nonfiction", "non-fiction")
        .otherwise(col("shelf_name"))
    )

    # Aggregate
    genre_counts = df_genres.groupBy("read_year", "genre").count().orderBy("read_year")
    
    # Collect
    print(f"   Collecting results for {start_year}-{end_year}...")
    pdf_trends = genre_counts.toPandas()
    
    # Save
    filename = f"genre_trends_{start_year}_{end_year}.csv"
    if not pdf_trends.empty:
        pdf_trends.to_csv(filename, index=False)
        print(f"✅ Saved {filename}")
    else:
        print(f"⚠️ No data for {start_year}-{end_year}")

### Execution Loop
Running the analysis in chunks.

In [6]:
intervals = [
    (2010, 2015),
    (2015, 2020),
    (2020, 2025)
]

for start, end in intervals:
    process_genre_interval(df_with_year, start, end)
    
print("\n🎉 All intervals processed.")


⏳ Processing Interval: 2010-2015


✅ Saved genre_trends_2010_2015.csv

⏳ Processing Interval: 2015-2020


✅ Saved genre_trends_2015_2020.csv

⏳ Processing Interval: 2020-2025


✅ Saved genre_trends_2020_2025.csv

🎉 All intervals processed.
